# YOLO Test - All classes 성능

In [ ]:
import torch
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5l')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-1 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


* 6 classes generator 생성

In [ ]:
def create_image_generator(target_class_name, target_count=200, other_count=400, batch_size=32, img_size=640):
    # 클래스 라벨링
    class_labels = {
        'Bicycle': 1,
        'Car': 2,
        'Motorcycle': 3,
        'Bus': 5,
        'Traffic Light': 9,
        'Hydrant': 10,
    }

    base_dir = '/content/drive/MyDrive/Kaggle/Input/Recaptcha_Dataset/images/'

    # 특정 클래스의 이미지 경로들
    target_class_dir = os.path.join(base_dir, target_class_name)
    target_image_paths = [(os.path.join(target_class_dir, fname), class_labels[target_class_name]) for fname in os.listdir(target_class_dir) if fname.endswith('.png')]
    random.shuffle(target_image_paths)
    target_image_paths = target_image_paths[:target_count]

    # 특정 클래스가 아닌 이미지 경로들
    other_image_paths = []
    for class_name, label in class_labels.items():
        if class_name != target_class_name:
            dir = os.path.join(base_dir, class_name)
            other_image_paths.extend([(os.path.join(dir, fname), label) for fname in os.listdir(dir) if fname.endswith('.png')])
    random.shuffle(other_image_paths)
    other_image_paths = other_image_paths[:other_count]

    # 두 목록 합치기
    all_image_paths = target_image_paths + other_image_paths
    random.shuffle(all_image_paths)

    # 이미지와 라벨을 배치로 로딩하고 반환하는 generator
    for i in range(0, len(all_image_paths), batch_size):
        batch_paths = all_image_paths[i: i + batch_size]
        batch_images = [cv2.imread(path[0]) for path in batch_paths]
        batch_labels = [path[1] for path in batch_paths]
        batch_images = [cv2.resize(img, (img_size, img_size)) for img in batch_images if img is not None]
        yield batch_images, batch_labels

class_labels = {
    'Bicycle': 1,
    'Car': 2,
    'Motorcycle': 3,
    'Bus': 5,
    'Traffic Light': 9,
    'Hydrant': 10,
}

* 6 classes classification report

In [ ]:
from sklearn.metrics import classification_report
from collections import Counter

def binary_classification_report_for_class(target_class, generator, yolo_model):
    true_counts = Counter()
    predicted_counts = Counter()

    # 실제 라벨과 예측 라벨을 담을 리스트
    true_label_list = []
    predicted_label_list = []

    # 모든 이미지에 대해
    for batch_images, true_labels in generator:
        # 이미지를 텐서로 변환
        batch_tensor = torch.tensor(batch_images).permute(0, 3, 1, 2).float() / 255.0

        # GPU 사용
        if torch.cuda.is_available():
            batch_tensor = batch_tensor.cuda()

        with torch.no_grad():
            predictions = yolo_model(batch_tensor)

        conf_threshold = 0.3
        mask = predictions[..., 4] > conf_threshold

        for i, img_predictions in enumerate(predictions):
            img_predictions = img_predictions[mask[i]]
            _, predicted_classes = img_predictions[:, 5:].max(1)

            # 예측된 bbox가 있다면 가장 확률이 높은 클래스 선택
            if img_predictions.size(0) != 0:
                max_conf, max_class_idx = img_predictions[:, 5:].max(1)
                predicted_class = max_class_idx[max_conf.argmax()].item()
                predicted_counts[predicted_class] += 1

                # 이진 분류를 위한 예측 라벨 처리
                predicted_bin_label = 1 if predicted_class == target_class else 0
                predicted_label_list.append(predicted_bin_label)
            else:
                # 예측되지 않은 경우 (예측 라벨이 없는 경우) 0으로 처리
                predicted_label_list.append(0)

            # 실제 라벨 카운트
            true_counts[true_labels[i]] += 1

            # 이진 분류를 위한 실제 라벨 처리
            true_bin_label = 1 if true_labels[i] == target_class else 0
            true_label_list.append(true_bin_label)

    # 이진 분류 성능 보고서 반환
    labels = [0, 1]
    target_names = [f'Not Class {target_class}', f'Class {target_class}']
    report = classification_report(true_label_list, predicted_label_list, labels=labels, target_names=target_names)
    return report

# 클래스 라벨링
class_labels = {
    'Bicycle': 1,
    'Car': 2,
    'Motorcycle': 3,
    'Bus': 5,
    'Traffic Light': 9,
    'Hydrant': 10,
}

In [ ]:
for class_name in class_labels.keys():
    generator_name = f"{class_name.lower().replace(' ', '_')}_generator"
    locals()[generator_name] = create_image_generator(class_name)

    labels = []
    for images, batch_labels in locals()[generator_name]:
        labels.extend(batch_labels)

    counter = Counter(labels)

    print(f"\nCounts for {class_name}:")
    for label, count in sorted(counter.items()):
        print(f"Label {label}: {count} images")

for class_name, class_id in class_labels.items():
    generator_name = f"{class_name.lower().replace(' ', '_')}_generator"
    generator_instance = create_image_generator(class_name)
    print(f"Performance report for {class_name} (Class {class_id}):")
    print(binary_classification_report_for_class(class_id, generator_instance, yolo_model))
    print("="*50)



Counts for Bicycle:
Label 1: 200 images
Label 2: 207 images
Label 3: 8 images
Label 5: 76 images
Label 9: 52 images
Label 10: 57 images

Counts for Car:
Label 1: 84 images
Label 2: 200 images
Label 3: 12 images
Label 5: 109 images
Label 9: 96 images
Label 10: 99 images

Counts for Motorcycle:
Label 1: 46 images
Label 2: 180 images
Label 3: 81 images
Label 5: 71 images
Label 9: 45 images
Label 10: 58 images

Counts for Bus:
Label 1: 54 images
Label 2: 219 images
Label 3: 3 images
Label 5: 200 images
Label 9: 53 images
Label 10: 71 images

Counts for Traffic Light:
Label 1: 39 images
Label 2: 215 images
Label 3: 8 images
Label 5: 83 images
Label 9: 200 images
Label 10: 55 images

Counts for Hydrant:
Label 1: 56 images
Label 2: 211 images
Label 3: 6 images
Label 5: 78 images
Label 9: 49 images
Label 10: 200 images
Performance report for Bicycle (Class 1):
              precision    recall  f1-score   support

 Not Class 1       0.75      0.99      0.86       400
     Class 1       0.97  

| Class          | Accuracy | F1-score (Target Class) | F1-score (Non-Target Class) | Count |
|----------------|----------|-------------------------|-----------------------------|-------|
| Bicycle        |   0.78   |      0.51               |      0.86                   | 600   |
| Bus            |   0.76   |      0.44               |      0.85                   | 600   |
| Car            |   0.76   |      0.51               |      0.84                   | 600   |
| Hydrant        |   0.92   |      0.86               |      0.94                   | 600   |
| Motorcycle     |   0.88   |      0.42               |      0.93                   | 241   |
| Traffic Light  |   0.81   |      0.66               |      0.87                   | 600   |
| Average Total  |   0.81   |      0.56               |      0.88                   | 6051  |